In [3]:
# new code, the buy_range and sell_range are calculated based on Somos prices

In [4]:
import yfinance as yf
import pandas as pd
from datetime import datetime

# S&P 50 stocks
stock_data = pd.read_csv("US1313.csv", encoding='latin-1')
stock_data = stock_data["Symbol"]

# Convert the DataFrame to a list
stock_data = stock_data.values.tolist()

# Adjustable threshold for anomaly detection
anomaly_threshold = 0.15  # 15% threshold, adjust as needed

def fetch_data(ticker, start_date, end_date, interval="1d"):
    try:
        return yf.download(ticker, start=start_date, end=end_date)
    except Exception as e:
        print(f"Failed to download data for {ticker}: {e}")
        return None

def generate_somos_sequence(data):
    somos_sequence = []
    for i in range(3, len(data)):
        term = (data[i-1] * data[i-2] + data[i-2] * data[i-3] + data[i-3] * data[i-1]) / data[i-3]
        somos_sequence.append(term)
    return somos_sequence

def detect_anomalies(dates, actual_data, somos_data, threshold):
    anomalies = []
    for i in range(len(somos_data)):
        relative_difference = abs(actual_data[i+3] - somos_data[i]) / actual_data[i+3]
        if relative_difference > threshold:
            anomalies.append((dates[i+3], actual_data[i+3], somos_data[i], relative_difference))
    return anomalies

def calculate_buy_sell_ranges(somos_sequence, buy_range_factor=0.85, sell_range_factor=1.10):
    if not somos_sequence:
        return None, None
    somos_min = min(somos_sequence)
    somos_max = max(somos_sequence)
    buy_range = (somos_min * buy_range_factor, somos_max * buy_range_factor)
    sell_range = (somos_min * sell_range_factor, somos_max * sell_range_factor)
    return buy_range, sell_range

def generate_signals(data, somos_sequence, buy_range, sell_range):
    if buy_range is None or sell_range is None:
        return [], False, None, None
    
    signals = []
    holding = False
    last_buy_date, last_buy_price = None, None

    for i in range(len(somos_sequence)):
        date = data.index[i + 3]
        actual_price = data['Close'].iloc[i + 3]
        somos_price = somos_sequence[i]

        if not holding and buy_range[0] <= somos_price <= buy_range[1]:
            last_buy_price = actual_price
            last_buy_date = date
            holding = True
        elif holding and sell_range[0] <= somos_price <= sell_range[1]:
            sell_price = actual_price
            sell_date = date
            holding = False
            signals.append((last_buy_date, last_buy_price, sell_date, sell_price))
    
    return signals, holding, last_buy_date, last_buy_price

def calculate_metrics(signals, initial_portfolio_value, holding, last_buy_date, last_buy_price):
    summary = []
    portfolio_value = initial_portfolio_value
    max_portfolio_value = initial_portfolio_value

    for signal in signals:
        buy_date, buy_price, sell_date, sell_price = signal
        shares_bought = portfolio_value / buy_price
        portfolio_value = shares_bought * sell_price
        max_portfolio_value = max(max_portfolio_value, portfolio_value)
        drawdown = (max_portfolio_value - portfolio_value) / max_portfolio_value
        percent_change = (sell_price - buy_price) / buy_price
        profit_percent = (portfolio_value - initial_portfolio_value) / initial_portfolio_value*100
        summary.append((buy_date, buy_price, sell_date, sell_price, percent_change, profit_percent, portfolio_value, drawdown))
    
    if holding:
        summary.append((last_buy_date, last_buy_price, "N/A", "N/A", "N/A", "N/A", portfolio_value, "Long Position Open"))
    return summary

def main():
    start_date = '2023-01-01'
    end_date = '2024-01-03'
    interval = "1d"
    
    open_position_dfs = []

    for ticker in stock_data:
        aapl_data = fetch_data(ticker, start_date, end_date, interval)
        
        if aapl_data is not None:
            closing_prices = aapl_data['Close'].tolist()
            somos_sequence = generate_somos_sequence(closing_prices)
            buy_range, sell_range = calculate_buy_sell_ranges(somos_sequence)
            signals, holding, last_buy_date, last_buy_price = generate_signals(aapl_data, somos_sequence, buy_range, sell_range)
            summary_table = calculate_metrics(signals, 100000, holding, last_buy_date, last_buy_price)
            
            if holding:
                summary_df = pd.DataFrame(summary_table, columns=['Buy Date', 'Buy Price', 'Sell Date', 'Sell Price', '% Change', '% Profit', 'Portfolio Value', 'Drawdown/Position'])
                summary_df['Ticker'] = ticker
                open_position_dfs.append(summary_df)

    if open_position_dfs:
        current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        csv_file_name = f'open_positions_{current_time}.csv'
        combined_df = pd.concat(open_position_dfs, ignore_index=True)
        combined_df.to_csv(csv_file_name, index=False)
        print("Saved open positions to:", csv_file_name)
    else:
        print("No open positions found.")

if __name__ == "__main__":
    main()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******


1 Failed download:
['RETA']: Exception('%ticker%: No data found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

...


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******


1 Failed download:
['CIR']: Exception('%ticker%: No data found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******


1 Failed download:
['CRHCF']: Exception('%ticker%: No data found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******